In [ ]:
# Spain data ingestion
import os
import geopandas as gpd

# Constants
WSG_84 = 4326
SPAIN_CRS = 3042

# Params - paths
silver_dir_path = "/opt/data/silver"

spain_shape_landing = "/opt/data/landing/spain_shape/SHP_ETRS89/recintos_provinciales_inspire_peninbal_etrs89/recintos_provinciales_inspire_peninbal_etrs89.shp"
spain_contour_path = f"{silver_dir_path}/spain/spain_contour.parquet"

# Ingestion
spain_shape = gpd.read_file(spain_shape_landing).to_crs(SPAIN_CRS)

# Transform
wanted_cols = ["INSPIREID", "geometry"]
spain_contour = spain_shape[wanted_cols].dissolve()

# Write
os.makedirs(os.path.dirname(spain_contour_path), exist_ok=True)
spain_contour.to_parquet(spain_contour_path)

In [ ]:
gpd.read_parquet(spain_contour_path).head()

# H3 Hexagons

In [ ]:
import geopandas as gpd
from shapely.geometry import shape, Polygon, MultiPolygon
import h3
import json

# Path to data
silver_dir_path = "/opt/data/silver"
spain_contour_path = f"{silver_dir_path}/spain/spain_contour.parquet"

# Load the shapefile containing Spain's contour
gdf = gpd.read_parquet(spain_contour_path)

# Ensure it has one row containing a multipolygon
spain_geometry = gdf.iloc[0].geometry

# Define the H3 resolution (the higher the number, the finer the hexagons)
resolution = 5

# Function to get the H3 hexagons completely within the multipolygon
def get_h3_hexagons_within(geometry, resolution):
    hexagons = set()
    if geometry.geom_type == 'MultiPolygon':
        for polygon in geometry.geoms:
            hexagons.update(get_h3_hexagons_within_polygon(polygon, resolution))
    elif geometry.geom_type == 'Polygon':
        hexagons.update(get_h3_hexagons_within_polygon(geometry, resolution))
    return list(hexagons)

def get_h3_hexagons_within_polygon(polygon, resolution):
    hexagons = set()
    # Get the bounding box of the polygon
    minx, miny, maxx, maxy = polygon.bounds
    # Iterate over the bounding box area with a step size to cover the polygon
    for lat in range(int(miny), int(maxy)+1):
        for lng in range(int(minx), int(maxx)+1):
            hex_id = h3.geo_to_h3(lat, lng, resolution)
            hex_boundary = h3.h3_to_geo_boundary(hex_id, geo_json=True)
            hex_polygon = shape({'type': 'Polygon', 'coordinates': [hex_boundary]})
            # Check if the hexagon is completely within the polygon
            if polygon.contains(hex_polygon):
                hexagons.add(hex_id)
    return hexagons

# Get the H3 hexagons within the multipolygon of Spain
h3_hexagons = get_h3_hexagons_within(spain_geometry, resolution)

# Convert H3 hexagons to polygons for visualization
hexagons_polygons = [Polygon(h3.h3_to_geo_boundary(hex_id, geo_json=True)) for hex_id in h3_hexagons]

# Create a GeoDataFrame from the hexagon polygons
hex_gdf = gpd.GeoDataFrame({'h3_id': h3_hexagons, 'geometry': hexagons_polygons})

# Save the result to a new shapefile (optional)
# hex_gdf.to_file('spain_h3_grid.shp')

# Visualize (optional)
# hex_gdf.explore()
